In [1]:
import pandas as pd

from pandarallel import pandarallel

from freeholdforecast.tasks.etl_ml_task import ETL_ML_Task

pd.options.display.max_columns = 100

task = ETL_ML_Task()


2022-09-07 00:00:24 INFO ETL_ML_Task Initialized task for ohio-hamilton with run date 2016-09-01


In [2]:
task._get_df_raw_encoded()
# task.df_raw_encoded


2022-09-07 00:00:24 INFO ETL_ML_Task Retrieving and encoding data
2022-09-07 00:00:24 INFO ETL_ML_Task Loading existing raw data
2022-09-07 00:00:26 INFO ETL_ML_Task Total parcels: 280902
2022-09-07 00:00:26 INFO ETL_ML_Task Total sales: 497550
2022-09-07 00:00:26 INFO ETL_ML_Task Loading existing encoded data


In [3]:
task._get_df_prepared()
# task.df_prepared


2022-09-07 00:00:26 INFO ETL_ML_Task Loading existing prepared data
2022-09-07 00:00:27 INFO ETL_ML_Task Splitting data
2022-09-07 00:00:27 INFO ETL_ML_Task Train dates: 2006-08-01 to 2016-08-01
2022-09-07 00:00:27 INFO ETL_ML_Task Test dates: 2016-09-01 to 2018-09-01


In [4]:
label_names = [
    "transfer_in_3_months",
    "transfer_in_6_months",
    "transfer_in_12_months",
    "transfer_in_24_months",
]

task.df_prepared.groupby(label_names).size()

transfer_in_3_months  transfer_in_6_months  transfer_in_12_months  transfer_in_24_months
0                     0                     0                      0                        4731482
                                                                   1                         175232
                                            1                      1                          99134
                      1                     1                      1                          55025
1                     1                     1                      1                          60391
dtype: int64

In [ ]:
task._train_models()

2022-09-07 00:00:28 INFO ETL_ML_Task Total labels: 4
2022-09-07 00:00:28 INFO ETL_ML_Task Jobs per label: 2
2022-09-07 00:00:28 INFO ETL_ML_Task Total fit minutes: 15
2022-09-07 00:00:28 INFO ETL_ML_Task Job fit minutes: 10
2022-09-07 00:00:28 INFO ETL_ML_Task Job memory limit (MB): 4096
2022-09-07 00:00:28 INFO ETL_ML_Task ----------------------  Start init AutoML for transfer_in_3_months ----------------------
2022-09-07 00:00:29 INFO ETL_ML_Task Train labels: 28758/2435461 (1.18%)
2022-09-07 00:00:29 INFO ETL_ML_Task Train res labels: 28758/316338 (9.09%)
2022-09-07 00:00:29 INFO ETL_ML_Task Test labels: 8364/619404 (1.35%)
2022-09-07 00:00:29 INFO ETL_ML_Task ----------------------   End init AutoML for transfer_in_3_months  ----------------------
2022-09-07 00:00:29 INFO ETL_ML_Task ----------------------  Start init AutoML for transfer_in_6_months ----------------------
2022-09-07 00:00:30 INFO ETL_ML_Task Train labels: 54497/2435461 (2.24%)
2022-09-07 00:00:30 INFO ETL_ML_Task T

/home/vscode/.local/share/virtualenvs/freehold-forecast-zrIN4e5_/lib/python3.8/site-packages/distributed/node.py:183: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37129 instead
  warnings.warn(


[WARNING] [2022-09-07 00:01:43,181:Client-AutoML(1):3c98da01-2e40-11ed-908d-0242ac110002] Capping the per_run_time_limit to 448.0 to have time for a least 2 models in each process.
2022-09-07 00:02:04 INFO ETL_ML_Task Fitting model for transfer_in_12_months
[WARNING] [2022-09-07 00:02:06,976:Client-AutoML(1):4e8d75a6-2e40-11ed-9194-0242ac110002] Dataset too large for memory limit 4096MB, reducing the precision from float64 to <class 'numpy.float32'>
[WARNING] [2022-09-07 00:02:07,119:Client-AutoML(1):4e8d75a6-2e40-11ed-9194-0242ac110002] Dataset too large for memory limit 4096MB, reducing number of samples from 2070156 to 1851279.


/home/vscode/.local/share/virtualenvs/freehold-forecast-zrIN4e5_/lib/python3.8/site-packages/distributed/node.py:183: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 38425 instead
  warnings.warn(


[WARNING] [2022-09-07 00:02:11,381:Client-AutoML(1):4e8d75a6-2e40-11ed-9194-0242ac110002] Capping the per_run_time_limit to 448.0 to have time for a least 2 models in each process.
2022-09-07 00:02:35 INFO ETL_ML_Task Fitting model for transfer_in_24_months
[WARNING] [2022-09-07 00:02:39,164:Client-AutoML(1):60b8245c-2e40-11ed-92b6-0242ac110002] Dataset too large for memory limit 4096MB, reducing the precision from float64 to <class 'numpy.float32'>
[WARNING] [2022-09-07 00:02:39,358:Client-AutoML(1):60b8245c-2e40-11ed-92b6-0242ac110002] Dataset too large for memory limit 4096MB, reducing number of samples from 2070156 to 1851279.


/home/vscode/.local/share/virtualenvs/freehold-forecast-zrIN4e5_/lib/python3.8/site-packages/distributed/node.py:183: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 40705 instead
  warnings.warn(


[WARNING] [2022-09-07 00:02:44,888:Client-AutoML(1):60b8245c-2e40-11ed-92b6-0242ac110002] Capping the per_run_time_limit to 448.0 to have time for a least 2 models in each process.
